# Function Calling with OpenAI's GPT Models

In [17]:
import langchain
import openai
import json
import ast

import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

## OpenAI Vanilla Example

In [2]:
function_descriptions = [
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {
                            "type": "string",
                            "description": "The temperature unit to use. Infer this from the users location.",
                            "enum": ["celsius", "fahrenheit"]
                        },
                    },
                    "required": ["location", "unit"],
                },
            }
        ]

In [3]:
user_query = "What's the weather like in San Francisco?"

Note: `function_call="auto"` will allow the model to choose whether or not it responds with a function.

In [15]:
response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo-0613',
    messages=[{'role': 'user', 'content': user_query}],
    functions=function_descriptions,
    function_call='auto'
)

In [16]:
ai_response_message = response['choices'][0]['message']
print(ai_response_message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"San Francisco\",\n  \"unit\": \"celsius\"\n}"
  }
}


In [9]:
# Cleaning up user response
user_location = ast.literal_eval(ai_response_message['function_call']['arguments']).get('location')
user_unit = ast.literal_eval(ai_response_message['function_call']['arguments']).get('unit')

In [10]:
def get_current_weather(location, unit):
    """Get the current weather in a given location"""

    weather_info = {
        'location': location,
        'temperature': '72',
        'unit': unit,
        'forecast': ['sunny', 'windy']
    }

    return json.dumps(weather_info)

In [11]:
function_response = get_current_weather(
    location=user_location,
    unit=user_unit
)

In [12]:
function_response

'{"location": "San Francisco, CA", "temperature": "72", "unit": "celsius", "forecast": ["sunny", "windy"]}'

In [13]:
# Pass the response to our model for a natural language response

second_response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo-0613',
    messages=[
        {'role': 'user', 'content': user_query},
        ai_response_message,
        {
            'role': 'function',
            'name': 'get_current_weather',
            'content': function_response
        }
    ]
)

In [14]:
print(second_response['choices'][0]['message']['content'])

The current weather in San Francisco is sunny and windy with a temperature of 72°C.


## Langchain Support For Functions